In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_columns=200

from preprocessing import Processor

sensors = [
    'abpd',
    'abps',
    'heartrate',
    '{SENSOR}',
    'temperature'
]

SENSOR = "temperature"

# Import Dataset

In [2]:
df_train = pd.read_parquet(f'../data/experiment4/wrangle/sensor_seed_merged/train/{SENSOR}.parquet')
df_test = pd.read_parquet(f'../data/experiment4/wrangle/sensor_seed_merged/test/{SENSOR}.parquet')

## Train

In [3]:
df_train.shape

(14972, 140)

In [4]:
df_train.head()

,temperature_trace_no,temperature_collected,temperature_collected_started,temperature_collected_started_relative,temperature_collected_finished,temperature_collected_finished_relative,temperature_battery_became_unavailable_during_collected,temperature_error_reached_during_collected,temperature_became_unavailable_during_collected,temperature_was_available_during_collected,temperature_collected_duration,temperature_battery_decrease_during_collected,temperature_processed,temperature_processed_started,temperature_processed_started_relative,temperature_processed_finished,temperature_processed_finished_relative,temperature_battery_became_unavailable_during_processed,temperature_error_reached_during_processed,temperature_became_unavailable_during_processed,temperature_was_available_during_processed,temperature_processed_duration,temperature_battery_decrease_during_processed,temperature_transfered,temperature_transfered_started,temperature_transfered_started_relative,temperature_transfered_finished,temperature_transfered_finished_relative,temperature_battery_became_unavailable_during_transfered,temperature_error_reached_during_transfered,temperature_became_unavailable_during_transfered,temperature_was_available_during_transfered,temperature_transfered_duration,temperature_battery_decrease_during_transfered,centralhub_temperature_processed,centralhub_temperature_processed_started,centralhub_temperature_processed_started_relative,centralhub_temperature_processed_finished,centralhub_temperature_processed_finished_relative,centralhub_temperature_processed_duration,temperature_battery_became_unavailable_during_centralhub_processed,temperature_error_reached_during_centralhub_processed,temperature_became_unavailable_during_centralhub_processed,temperature_was_available_during_centralhub_processed,centralhub_battery_became_unavailable_during_centralhub_processed,centralhub_detected,centralhub_detected_started,centralhub_detected_started_relative,centralhub_detected_finished,centralhub_detected_finished_relative,centralhub_detected_duration,temperature_battery_became_unavailable_during_centralhub_detected,temperature_error_reached_during_centralhub_detected,temperature_became_unavailable_during_centralhub_detected,temperature_was_available_during_centralhub_detected,centralhub_battery_became_unavailable_during_centralhub_detected,temperature_battery_decrease_during_trace,centralhub_battery_decrease_during_trace,centralhub_battery_became_unavailable_during_centralhub_temperature_processed,temperature_battery_became_unavailable_during_centralhub_temperature_processed,temperature_error_reached_during_centralhub_temperature_processed,temperature_became_unavailable_during_centralhub_temperature_processed,temperature_was_available_during_centralhub_temperature_processed,time_min,time_max,abpd_context_min,abps_context_min,heartrate_context_min,oximeter_context_min,temperature_context_min,abpd_context_became_unavailable_max,abps_context_became_unavailable_max,heartrate_context_became_unavailable_max,oximeter_context_became_unavailable_max,temperature_context_became_unavailable_max,abpd_measurement_max,abpd_measurement_mean,abpd_measurement_std,abpd_measurement_min,abps_measurement_max,abps_measurement_mean,abps_measurement_std,abps_measurement_min,heartrate_measurement_max,heartrate_measurement_mean,heartrate_measurement_std,heartrate_measurement_min,oximeter_measurement_max,oximeter_measurement_mean,oximeter_measurement_std,oximeter_measurement_min,temperature_measurement_max,temperature_measurement_mean,temperature_measurement_std,temperature_measurement_min,abpd_sig_battery_on_min,abps_sig_battery_on_min,heartrate_sig_battery_on_min,oximeter_sig_battery_on_min,temperature_sig_battery_on_min,abpd_sig_battery_on_became_unavailable_max,abps_sig_battery_on_became_unavailable_max,heartrate_sig_battery_on_became_unavailable_max,oximeter_sig_battery_on_became_unavailable_max,temperature_sig_battery_on_became_unavailable_max,abpd_battery_cell_soc_max,abpd_

## Test

In [5]:
df_test.shape

(5024, 140)

In [6]:
df_test.head()

,temperature_trace_no,temperature_collected,temperature_collected_started,temperature_collected_started_relative,temperature_collected_finished,temperature_collected_finished_relative,temperature_battery_became_unavailable_during_collected,temperature_error_reached_during_collected,temperature_became_unavailable_during_collected,temperature_was_available_during_collected,temperature_collected_duration,temperature_battery_decrease_during_collected,temperature_processed,temperature_processed_started,temperature_processed_started_relative,temperature_processed_finished,temperature_processed_finished_relative,temperature_battery_became_unavailable_during_processed,temperature_error_reached_during_processed,temperature_became_unavailable_during_processed,temperature_was_available_during_processed,temperature_processed_duration,temperature_battery_decrease_during_processed,temperature_transfered,temperature_transfered_started,temperature_transfered_started_relative,temperature_transfered_finished,temperature_transfered_finished_relative,temperature_battery_became_unavailable_during_transfered,temperature_error_reached_during_transfered,temperature_became_unavailable_during_transfered,temperature_was_available_during_transfered,temperature_transfered_duration,temperature_battery_decrease_during_transfered,centralhub_temperature_processed,centralhub_temperature_processed_started,centralhub_temperature_processed_started_relative,centralhub_temperature_processed_finished,centralhub_temperature_processed_finished_relative,centralhub_temperature_processed_duration,temperature_battery_became_unavailable_during_centralhub_processed,temperature_error_reached_during_centralhub_processed,temperature_became_unavailable_during_centralhub_processed,temperature_was_available_during_centralhub_processed,centralhub_battery_became_unavailable_during_centralhub_processed,centralhub_detected,centralhub_detected_started,centralhub_detected_started_relative,centralhub_detected_finished,centralhub_detected_finished_relative,centralhub_detected_duration,temperature_battery_became_unavailable_during_centralhub_detected,temperature_error_reached_during_centralhub_detected,temperature_became_unavailable_during_centralhub_detected,temperature_was_available_during_centralhub_detected,centralhub_battery_became_unavailable_during_centralhub_detected,temperature_battery_decrease_during_trace,centralhub_battery_decrease_during_trace,centralhub_battery_became_unavailable_during_centralhub_temperature_processed,temperature_battery_became_unavailable_during_centralhub_temperature_processed,temperature_error_reached_during_centralhub_temperature_processed,temperature_became_unavailable_during_centralhub_temperature_processed,temperature_was_available_during_centralhub_temperature_processed,time_min,time_max,abpd_context_min,abps_context_min,heartrate_context_min,oximeter_context_min,temperature_context_min,abpd_context_became_unavailable_max,abps_context_became_unavailable_max,heartrate_context_became_unavailable_max,oximeter_context_became_unavailable_max,temperature_context_became_unavailable_max,abpd_measurement_max,abpd_measurement_mean,abpd_measurement_std,abpd_measurement_min,abps_measurement_max,abps_measurement_mean,abps_measurement_std,abps_measurement_min,heartrate_measurement_max,heartrate_measurement_mean,heartrate_measurement_std,heartrate_measurement_min,oximeter_measurement_max,oximeter_measurement_mean,oximeter_measurement_std,oximeter_measurement_min,temperature_measurement_max,temperature_measurement_mean,temperature_measurement_std,temperature_measurement_min,abpd_sig_battery_on_min,abps_sig_battery_on_min,heartrate_sig_battery_on_min,oximeter_sig_battery_on_min,temperature_sig_battery_on_min,abpd_sig_battery_on_became_unavailable_max,abps_sig_battery_on_became_unavailable_max,heartrate_sig_battery_on_became_unavailable_max,oximeter_sig_battery_on_became_unavailable_max,temperature_sig_battery_on_became_unavailable_max,abpd_battery_cell_soc_max,abpd_

# Failures

Both columns _is_property_violation_ and _is_failure_ columns are trying to capture anomalous behavior. They look at traces in which the sensor collected, but the Central Node did not process. 

The difference between them lies in the context of the the sensor. While the column _is_property_violation_ doesn't make a distinction betweeen the sensor being present or absent, the column _is_failure_ considers only the cases when the sensor was present.

This section checks whether the _is_failure_ column is the intended subset of the _is_property_violation_.

## Train

In [7]:
df_train.is_failure.value_counts()

0    14243
1      729
Name: is_failure, dtype: int64

In [8]:
(df_train
.query("is_property_violation == 1")
.groupby([
    f'{SENSOR}_collected', 
    f'{SENSOR}_processed',
    f'{SENSOR}_transfered',
    f'centralhub_{SENSOR}_processed',
    'is_failure',
])
.size()
)

temperature_collected  temperature_processed  temperature_transfered  centralhub_temperature_processed  is_failure
1                      0                      0                       0                                 0               62
                                                                                                        1              177
                       1                      0                       0                                 0             1361
                                                                                                        1              286
                                              1                       0                                 0               71
                                                                                                        1              266
dtype: int64

: 

For every combination of the main signals, when there is a property violation, we have cases where _is_failure_ is true and false.

In [8]:
df_train.groupby(['is_failure','is_property_violation']).size()

is_failure  is_property_violation
0           0                        12749
            1                         1494
1           1                          729
dtype: int64

The opposite does not happen. Whenever a failure is spotted, a property violation is also spotted.

## Test

In [37]:
df_test.is_failure.value_counts()

0    4782
1     242
Name: is_failure, dtype: int64

In [38]:
(df_test
.query("is_property_violation == 1")
.groupby([
    f'{SENSOR}_collected', 
    f'{SENSOR}_processed',
    f'{SENSOR}_transfered',
    f'centralhub_{SENSOR}_processed',
    'is_failure',
])
.size()
)

temperature_collected  temperature_processed  temperature_transfered  centralhub_temperature_processed  is_failure
1                      0                      0                       0                                 0              23
                                                                                                        1              59
                       1                      0                       0                                 0             491
                                                                                                        1             101
                                              1                       0                                 0              33
                                                                                                        1              82
dtype: int64

For every combination of the main signals, when there is a property violation, we have cases where _is_failure_ is true and false.

In [39]:
df_test.groupby(['is_failure','is_property_violation']).size()

is_failure  is_property_violation
0           0                        4235
            1                         547
1           1                         242
dtype: int64

The opposite does not happen. Whenever a failure is spotted, a property violation is also spotted.

# Property Violations

## Train

In [13]:
df_train.is_property_violation.value_counts()

0    12749
1     2223
Name: is_property_violation, dtype: int64

In [44]:
(df_train
.groupby([
    f'{SENSOR}_collected', 
    f'{SENSOR}_processed',
    f'{SENSOR}_transfered',
    f'centralhub_{SENSOR}_processed',
    'is_property_violation'
])
.size()
)

temperature_collected  temperature_processed  temperature_transfered  centralhub_temperature_processed  is_property_violation
1                      0                      0                       0                                 1                          239
                       1                      0                       0                                 1                         1647
                                              1                       0                                 1                          337
                                                                      1                                 0                        12749
dtype: int64

From the above output, we can see that
* No data is processed without being collected
* No data is transfered without being processed
* No data is processed by the central not without the sensor transfering it
* When the Central Node process the data, we don't have a property violation. 

In [42]:
(df_train
.groupby([
    f'{SENSOR}_collected', 
    f'{SENSOR}_processed',
    f'{SENSOR}_transfered',
    f'centralhub_{SENSOR}_processed',
    f'{SENSOR}_sig_battery_on_became_unavailable_max', # flag indicating if the battery became unavailable during the trace
    'is_property_violation'
])
.size()
)

temperature_collected  temperature_processed  temperature_transfered  centralhub_temperature_processed  temperature_sig_battery_on_became_unavailable_max  is_property_violation
1                      0                      0                       0                                 1                                                  1                          239
                       1                      0                       0                                 0                                                  1                         1378
                                                                                                        1                                                  1                          269
                                              1                       0                                 0                                                  1                           95
                                                                               

From the above output, we can note that:
* Some of the property violations happen when the battery becomes unavailable. 
* In the traces in which the Central Node process the data, the sensor's battery does not run out

## Test

In [40]:
df_test.is_property_violation.value_counts()

0    4235
1     789
Name: is_property_violation, dtype: int64

In [43]:
(df_test
.groupby([
    f'{SENSOR}_collected', 
    f'{SENSOR}_processed',
    f'{SENSOR}_transfered',
    f'centralhub_{SENSOR}_processed',
    'is_property_violation'
])
.size()
)

temperature_collected  temperature_processed  temperature_transfered  centralhub_temperature_processed  is_property_violation
1                      0                      0                       0                                 1                          82
                       1                      0                       0                                 1                         592
                                              1                       0                                 1                         115
                                                                      1                                 0                        4235
dtype: int64

From the above output, we can see that
* No data is processed without being collected
* No data is transfered without being processed
* No data is processed by the central not without the sensor transfering it
* When the Central Node process the data, we don't have a property violation. 

In [45]:
(df_train
.groupby([
    f'{SENSOR}_collected', 
    f'{SENSOR}_processed',
    f'{SENSOR}_transfered',
    f'centralhub_{SENSOR}_processed',
    f'{SENSOR}_sig_battery_on_became_unavailable_max', # flag indicating if the battery became unavailable during the trace
    'is_property_violation'
])
.size()
)

temperature_collected  temperature_processed  temperature_transfered  centralhub_temperature_processed  temperature_sig_battery_on_became_unavailable_max  is_property_violation
1                      0                      0                       0                                 1                                                  1                          239
                       1                      0                       0                                 0                                                  1                         1378
                                                                                                        1                                                  1                          269
                                              1                       0                                 0                                                  1                           95
                                                                               

From the above output, we can note that:
* Some of the property violations happen when the battery becomes unavailable. 
* In the traces in which the Central Node process the data, the sensor's battery does not run out

# Battery

## Train

### Battery Ran out twice in the same trace

#### Battery Ran out during collected and processed

In [46]:
(df_train
.query(f"{SENSOR}_collected == 1")
.query(f'{SENSOR}_battery_became_unavailable_during_collected == 1')
.query(f"{SENSOR}_processed == 1")
# .query("{SENSOR}_transfered == 0")
.filter([
    f'{SENSOR}_collected',
    f'{SENSOR}_battery_became_unavailable_during_collected',
    f'{SENSOR}_processed',
    f'{SENSOR}_battery_became_unavailable_during_processed',
    f'{SENSOR}_transfered',
    f'{SENSOR}_battery_became_unavailable_during_transfered',
    f'centralhub_{SENSOR}_processed',
    f'is_property_violation',
    f'{SENSOR}_trace_no',
    'seed'
])
.T
)

""
temperature_collected
temperature_battery_became_unavailable_during_collected
temperature_processed
temperature_battery_became_unavailable_during_processed
temperature_transfered
temperature_battery_became_unavailable_during_transfered
centralhub_temperature_processed
is_property_violation
temperature_trace_no
seed


#### Battery Ran out during processed and transfered

In [47]:
(df_train
.query(f"{SENSOR}_processed == 1")
.query(f'{SENSOR}_battery_became_unavailable_during_processed == 1')
.query(f"{SENSOR}_transfered == 1")
# .query("{SENSOR}_transfered == 0")
.filter([
    f'{SENSOR}_collected',
    f'{SENSOR}_battery_became_unavailable_during_collected',
    f'{SENSOR}_processed',
    f'{SENSOR}_battery_became_unavailable_during_processed',
    f'{SENSOR}_transfered',
    f'{SENSOR}_battery_became_unavailable_during_transfered',
    f'centralhub_{SENSOR}_processed',
    f'is_property_violation',
    f'{SENSOR}_trace_no',
    'seed'
])
.T
)

""
temperature_collected
temperature_battery_became_unavailable_during_collected
temperature_processed
temperature_battery_became_unavailable_during_processed
temperature_transfered
temperature_battery_became_unavailable_during_transfered
centralhub_temperature_processed
is_property_violation
temperature_trace_no
seed


#### Overview

In [48]:
(df_train
.query(f'{SENSOR}_sig_battery_on_became_unavailable_max == 1')
.filter([
    f'{SENSOR}_battery_became_unavailable_during_collected',
    f'{SENSOR}_battery_became_unavailable_during_processed',
    f'{SENSOR}_battery_became_unavailable_during_transfered',
]
)).T

,13,28,54,80,102,120,140,162,188,210,236,252,268,295,311,334,352,378,395,415,441,458,484,511,537,565,581,600,617,632,652,667,684,700,725,742,764,782,801,818,839,856,872,895,910,927,944,964,990,1017,1037,1055,1074,1095,1117,1144,1160,1179,1195,1214,1234,1260,1283,1303,1320,1335,1361,1388,1413,1437,1455,1474,1495,1513,1533,1557,1572,1599,1621,1643,1661,1683,1701,1727,1746,1762,1788,1802,1819,1836,1857,1882,1910,1926,1942,1958,1977,1993,2008,2022,...,13036,13052,13067,13083,13098,13118,13138,13166,13184,13203,13222,13247,13273,13293,13313,13332,13354,13371,13387,13403,13427,13444,13462,13489,13516,13535,13550,13569,13584,13602,13620,13645,13660,13675,13699,13719,13736,13754,13774,13794,13813,13840,13865,13887,13906,13932,13953,13971,13987,14007,14028,14045,14070,14092,14109,14128,14146,14172,14190,14209,14225,14245,14260,14278,14296,14314,14330,14350,14374,14400,14415,14431,14452,14467,14493,14510,14534,14550,14575,14601,14621,14641,14657,14672,14688,14706,14730,14751,14773,14795,14811,14831,14848,14866,14881,14898,14914,14938,14956,14971
temperature_battery_became_unavailable_during_collected,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
temperature_battery_became_unavailable_during_processed,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,NaN,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,0.0,1.0,NaN,0.0,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,NaN,NaN,0.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0,0.0,NaN,NaN,1.0,0.0,1.0,NaN,NaN,1.0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,NaN,NaN,1.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,NaN,1.0,0.0,1.0,0.0,NaN,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,1.0,0.0,1.0,1.0,NaN,1.0,0.0,1.0,NaN,NaN,1.0,1.0,0.0,NaN,1.0,NaN,1.0,1.0,0.0
temperature_battery_became_unavailable_during_transfered,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0


* The sensor's battery becomes unavailable in only one of the processes.
* Whenever the battery runs out, it happens during one of the processes.
* Whenever the battery runs out, the next signals come empty.

### When battery had charge, are there records of the battery becoming unavailable?

In [49]:
(df_train
.query(f'{SENSOR}_sig_battery_on_min != 0')
.query(f'{SENSOR}_battery_became_unavailable_during_collected == 1 or {SENSOR}_battery_became_unavailable_during_processed == 1 or {SENSOR}_battery_became_unavailable_during_transfered == 1')
)

,temperature_trace_no,temperature_collected,temperature_collected_started,temperature_collected_started_relative,temperature_collected_finished,temperature_collected_finished_relative,temperature_battery_became_unavailable_during_collected,temperature_error_reached_during_collected,temperature_became_unavailable_during_collected,temperature_was_available_during_collected,temperature_collected_duration,temperature_battery_decrease_during_collected,temperature_processed,temperature_processed_started,temperature_processed_started_relative,temperature_processed_finished,temperature_processed_finished_relative,temperature_battery_became_unavailable_during_processed,temperature_error_reached_during_processed,temperature_became_unavailable_during_processed,temperature_was_available_during_processed,temperature_processed_duration,temperature_battery_decrease_during_processed,temperature_transfered,temperature_transfered_started,temperature_transfered_started_relative,temperature_transfered_finished,temperature_transfered_finished_relative,temperature_battery_became_unavailable_during_transfered,temperature_error_reached_during_transfered,temperature_became_unavailable_during_transfered,temperature_was_available_during_transfered,temperature_transfered_duration,temperature_battery_decrease_during_transfered,centralhub_temperature_processed,centralhub_temperature_processed_started,centralhub_temperature_processed_started_relative,centralhub_temperature_processed_finished,centralhub_temperature_processed_finished_relative,centralhub_temperature_processed_duration,temperature_battery_became_unavailable_during_centralhub_processed,temperature_error_reached_during_centralhub_processed,temperature_became_unavailable_during_centralhub_processed,temperature_was_available_during_centralhub_processed,centralhub_battery_became_unavailable_during_centralhub_processed,centralhub_detected,centralhub_detected_started,centralhub_detected_started_relative,centralhub_detected_finished,centralhub_detected_finished_relative,centralhub_detected_duration,temperature_battery_became_unavailable_during_centralhub_detected,temperature_error_reached_during_centralhub_detected,temperature_became_unavailable_during_centralhub_detected,temperature_was_available_during_centralhub_detected,centralhub_battery_became_unavailable_during_centralhub_detected,temperature_battery_decrease_during_trace,centralhub_battery_decrease_during_trace,centralhub_battery_became_unavailable_during_centralhub_temperature_processed,temperature_battery_became_unavailable_during_centralhub_temperature_processed,temperature_error_reached_during_centralhub_temperature_processed,temperature_became_unavailable_during_centralhub_temperature_processed,temperature_was_available_during_centralhub_temperature_processed,time_min,time_max,abpd_context_min,abps_context_min,heartrate_context_min,oximeter_context_min,temperature_context_min,abpd_context_became_unavailable_max,abps_context_became_unavailable_max,heartrate_context_became_unavailable_max,oximeter_context_became_unavailable_max,temperature_context_became_unavailable_max,abpd_measurement_max,abpd_measurement_mean,abpd_measurement_std,abpd_measurement_min,abps_measurement_max,abps_measurement_mean,abps_measurement_std,abps_measurement_min,heartrate_measurement_max,heartrate_measurement_mean,heartrate_measurement_std,heartrate_measurement_min,oximeter_measurement_max,oximeter_measurement_mean,oximeter_measurement_std,oximeter_measurement_min,temperature_measurement_max,temperature_measurement_mean,temperature_measurement_std,temperature_measurement_min,abpd_sig_battery_on_min,abps_sig_battery_on_min,heartrate_sig_battery_on_min,oximeter_sig_battery_on_min,temperature_sig_battery_on_min,abpd_sig_battery_on_became_unavailable_max,abps_sig_battery_on_became_unavailable_max,heartrate_sig_battery_on_became_unavailable_max,oximeter_sig_battery_on_became_unavailable_max,temperature_sig_battery_on_became_unavailable_max,abpd_battery_cell_soc_max,abpd_

When the battery had charge, we see no register of battery becoming unavailable

In [50]:
(df_train
.groupby([f'{SENSOR}_sig_battery_on_min',
f'{SENSOR}_sig_battery_on_became_unavailable_max',])
.size()
)

temperature_sig_battery_on_min  temperature_sig_battery_on_became_unavailable_max
0.0                             1                                                      750
1.0                             0                                                    14222
dtype: int64

These two columns have the same information. One of them will be dropped.

In [51]:
df_train.drop(columns=[f'{SENSOR}_sig_battery_on_min'], inplace=True)

### Overview of the signal battery unavailable

In [53]:
df_train.query(f'{SENSOR}_sig_battery_on_became_unavailable_max == 1')[f'{SENSOR}_battery_cell_soc_min'].describe()

count    750.000000
mean       0.094139
std        0.006029
min        0.074463
25%        0.089830
50%        0.095345
75%        0.100000
max        0.100000
Name: temperature_battery_cell_soc_min, dtype: float64

When we have an indication that the battery ran out, the state of charge is at maximum 0.1, aka the threshold.

## Test

### Battery Ran out twice in the same trace

#### Battery Ran out during collected and processed

In [54]:
(df_test
.query(f"{SENSOR}_collected == 1")
.query(f'{SENSOR}_battery_became_unavailable_during_collected == 1')
.query(f"{SENSOR}_processed == 1")
# .query("{SENSOR}_transfered == 0")
.filter([
    f'{SENSOR}_collected',
    f'{SENSOR}_battery_became_unavailable_during_collected',
    f'{SENSOR}_processed',
    f'{SENSOR}_battery_became_unavailable_during_processed',
    f'{SENSOR}_transfered',
    f'{SENSOR}_battery_became_unavailable_during_transfered',
    f'centralhub_{SENSOR}_processed',
    f'is_property_violation',
    f'{SENSOR}_trace_no',
    'seed'
])
.T
)

""
temperature_collected
temperature_battery_became_unavailable_during_collected
temperature_processed
temperature_battery_became_unavailable_during_processed
temperature_transfered
temperature_battery_became_unavailable_during_transfered
centralhub_temperature_processed
is_property_violation
temperature_trace_no
seed


#### Battery Ran out during processed and transfered

In [55]:
(df_test
.query(f"{SENSOR}_processed == 1")
.query(f'{SENSOR}_battery_became_unavailable_during_processed == 1')
.query(f"{SENSOR}_transfered == 1")
# .query("{SENSOR}_transfered == 0")
.filter([
    f'{SENSOR}_collected',
    f'{SENSOR}_battery_became_unavailable_during_collected',
    f'{SENSOR}_processed',
    f'{SENSOR}_battery_became_unavailable_during_processed',
    f'{SENSOR}_transfered',
    f'{SENSOR}_battery_became_unavailable_during_transfered',
    f'centralhub_{SENSOR}_processed',
    f'is_property_violation',
    f'{SENSOR}_trace_no',
    'seed'
])
.T
)

""
temperature_collected
temperature_battery_became_unavailable_during_collected
temperature_processed
temperature_battery_became_unavailable_during_processed
temperature_transfered
temperature_battery_became_unavailable_during_transfered
centralhub_temperature_processed
is_property_violation
temperature_trace_no
seed


#### Overview

In [56]:
(df_test
.query(f'{SENSOR}_sig_battery_on_became_unavailable_max == 1')
.filter([
    f'{SENSOR}_battery_became_unavailable_during_collected',
    f'{SENSOR}_battery_became_unavailable_during_processed',
    f'{SENSOR}_battery_became_unavailable_during_transfered',
]
)).T

,21,48,69,98,115,130,145,159,175,195,215,235,264,287,303,327,343,360,382,397,424,447,464,479,503,521,537,556,575,599,616,641,659,682,698,723,750,774,790,816,832,859,875,891,908,926,946,964,980,998,1023,1042,1069,1096,1111,1126,1147,1163,1192,1217,1239,1257,1278,1294,1318,1334,1357,1381,1407,1425,1445,1466,1486,1501,1522,1543,1558,1576,1592,1609,1628,1642,1665,1684,1707,1730,1750,1765,1782,1804,1822,1838,1859,1881,1899,1922,1945,1968,1989,2006,...,3030,3058,3076,3095,3116,3143,3159,3183,3202,3229,3253,3273,3289,3309,3330,3355,3373,3395,3415,3436,3456,3481,3497,3514,3534,3557,3580,3600,3626,3651,3672,3688,3714,3737,3754,3769,3791,3810,3833,3847,3863,3882,3896,3918,3933,3958,3972,3989,4005,4031,4053,4069,4085,4107,4130,4154,4178,4203,4220,4241,4265,4287,4303,4329,4346,4362,4380,4403,4423,4438,4456,4483,4500,4518,4532,4547,4567,4586,4601,4615,4642,4657,4675,4697,4723,4738,4756,4772,4796,4815,4840,4862,4878,4899,4917,4942,4969,4987,5008,5023
temperature_battery_became_unavailable_during_collected,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
temperature_battery_became_unavailable_during_processed,1.0,1.0,NaN,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,NaN,0.0,1.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,NaN,1.0,1.0,1.0,1.0,NaN,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,1.0,NaN,NaN,0.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,0.0,1.0,0.0,0.0,...,NaN,1.0,NaN,1.0,NaN,1.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,1.0,0.0,NaN,1.0,NaN,1.0,NaN,0.0,1.0,0.0,NaN,NaN,1.0,NaN,0.0,1.0,NaN,0.0,NaN,1.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,NaN,0.0,NaN,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0
temperature_battery_became_unavailable_during_transfered,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0


* The sensor's battery becomes unavailable in only one of the processes.
* Whenever the battery runs out, it happens during one of the processes.
* Whenever the battery runs out, the next signals come empty.

### When battery had charge, are there records of the battery becoming unavailable?

In [57]:
(df_test
.query(f'{SENSOR}_sig_battery_on_min != 0')
.query(f'{SENSOR}_battery_became_unavailable_during_collected == 1 or {SENSOR}_battery_became_unavailable_during_processed == 1 or {SENSOR}_battery_became_unavailable_during_transfered == 1')
)

,temperature_trace_no,temperature_collected,temperature_collected_started,temperature_collected_started_relative,temperature_collected_finished,temperature_collected_finished_relative,temperature_battery_became_unavailable_during_collected,temperature_error_reached_during_collected,temperature_became_unavailable_during_collected,temperature_was_available_during_collected,temperature_collected_duration,temperature_battery_decrease_during_collected,temperature_processed,temperature_processed_started,temperature_processed_started_relative,temperature_processed_finished,temperature_processed_finished_relative,temperature_battery_became_unavailable_during_processed,temperature_error_reached_during_processed,temperature_became_unavailable_during_processed,temperature_was_available_during_processed,temperature_processed_duration,temperature_battery_decrease_during_processed,temperature_transfered,temperature_transfered_started,temperature_transfered_started_relative,temperature_transfered_finished,temperature_transfered_finished_relative,temperature_battery_became_unavailable_during_transfered,temperature_error_reached_during_transfered,temperature_became_unavailable_during_transfered,temperature_was_available_during_transfered,temperature_transfered_duration,temperature_battery_decrease_during_transfered,centralhub_temperature_processed,centralhub_temperature_processed_started,centralhub_temperature_processed_started_relative,centralhub_temperature_processed_finished,centralhub_temperature_processed_finished_relative,centralhub_temperature_processed_duration,temperature_battery_became_unavailable_during_centralhub_processed,temperature_error_reached_during_centralhub_processed,temperature_became_unavailable_during_centralhub_processed,temperature_was_available_during_centralhub_processed,centralhub_battery_became_unavailable_during_centralhub_processed,centralhub_detected,centralhub_detected_started,centralhub_detected_started_relative,centralhub_detected_finished,centralhub_detected_finished_relative,centralhub_detected_duration,temperature_battery_became_unavailable_during_centralhub_detected,temperature_error_reached_during_centralhub_detected,temperature_became_unavailable_during_centralhub_detected,temperature_was_available_during_centralhub_detected,centralhub_battery_became_unavailable_during_centralhub_detected,temperature_battery_decrease_during_trace,centralhub_battery_decrease_during_trace,centralhub_battery_became_unavailable_during_centralhub_temperature_processed,temperature_battery_became_unavailable_during_centralhub_temperature_processed,temperature_error_reached_during_centralhub_temperature_processed,temperature_became_unavailable_during_centralhub_temperature_processed,temperature_was_available_during_centralhub_temperature_processed,time_min,time_max,abpd_context_min,abps_context_min,heartrate_context_min,oximeter_context_min,temperature_context_min,abpd_context_became_unavailable_max,abps_context_became_unavailable_max,heartrate_context_became_unavailable_max,oximeter_context_became_unavailable_max,temperature_context_became_unavailable_max,abpd_measurement_max,abpd_measurement_mean,abpd_measurement_std,abpd_measurement_min,abps_measurement_max,abps_measurement_mean,abps_measurement_std,abps_measurement_min,heartrate_measurement_max,heartrate_measurement_mean,heartrate_measurement_std,heartrate_measurement_min,oximeter_measurement_max,oximeter_measurement_mean,oximeter_measurement_std,oximeter_measurement_min,temperature_measurement_max,temperature_measurement_mean,temperature_measurement_std,temperature_measurement_min,abpd_sig_battery_on_min,abps_sig_battery_on_min,heartrate_sig_battery_on_min,oximeter_sig_battery_on_min,temperature_sig_battery_on_min,abpd_sig_battery_on_became_unavailable_max,abps_sig_battery_on_became_unavailable_max,heartrate_sig_battery_on_became_unavailable_max,oximeter_sig_battery_on_became_unavailable_max,temperature_sig_battery_on_became_unavailable_max,abpd_battery_cell_soc_max,abpd_

When the battery had charge, we see no register of battery becoming unavailable

In [58]:
(df_test
.groupby([f'{SENSOR}_sig_battery_on_min',
f'{SENSOR}_sig_battery_on_became_unavailable_max',])
.size()
)

temperature_sig_battery_on_min  temperature_sig_battery_on_became_unavailable_max
0.0                             1                                                     250
1.0                             0                                                    4774
dtype: int64

These two columns have the same information. One of them will be dropped.

In [59]:
df_test.drop(columns=[f'{SENSOR}_sig_battery_on_min'], inplace=True)

### Overview of the signal battery unavailable

In [60]:
df_test.query(f'{SENSOR}_sig_battery_on_became_unavailable_max == 1')[f'{SENSOR}_battery_cell_soc_min'].describe()

count    250.000000
mean       0.094062
std        0.006014
min        0.073296
25%        0.089838
50%        0.095391
75%        0.100000
max        0.100000
Name: temperature_battery_cell_soc_min, dtype: float64

When we have an indication that the battery ran out, the state of charge is at maximum 0.1, aka the threshold.